<a href="https://colab.research.google.com/github/laurence-lin/Coursera_Capstone/blob/master/Final_Project_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

import gc
import os
import requests

!pip install folium
!pip install geopy

import folium
from geopy.geocoders import Nominatim

from google.colab import files


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# CLIENT settings
CLIENT_ID = '02FTVAGTLJ1JCOACKPUG1FYI44KBDYCDVCYBT1BBZRPZ5PI2'
CLIENT_SECRET = 'IPSZRP2ZALOFZI2GCSCDP5UZPQWUDQ44XDTPKU3AQCTYTQPU'
VERSION = '20200603'

print('Foursquare parameters assigned.')


Foursquare parameters assigned.


In [1]:

files.upload()
!unzip *.zip

Saving zomato_restaurant.zip to zomato_restaurant.zip


{'zomato_restaurant.zip': b'PK\x03\x04-\x00\x00\x00\x08\x00z\xb3;O\xcc\x12b\x04\xff\xff\xff\xff\xff\xff\xff\xff\n\x00\x14\x00zomato.csv\x01\x00\x10\x00\xa4q"\x00\x00\x00\x00\x00\x93\xc5\x06\x00\x00\x00\x00\x00\xd4\xbd\xc9r\xe3H\xd6.\xb8\xbff\xf7\x1d\xd0\xb4\xb6\xca\xff\xdaE\xa8\xe0\x13\xe0\xe8UK\x8aPLR\x84*\xa4\xca\xb0\xbf6e\x10\x89\x90P\xa2\x08\x15HF\xa4r\xd5\xbd\xef\xa7\xe8\xbb\xaa\xc7\xe8\xd5}\x93\xfeW\xf1\x0e}>\x1f@L\xa48(\xbb\xfb\xa6UeR\x18H\xff\xdc\x8f\x9f\xc9\xcf\xf0%\x9f/\xb2e\x95\xcd\x16\xc1\xfb\xd7a\xe3\xafO\xd9C\x1e\x9e\x96\xcb\xd9\xa2z\nN\xcb\t\xfdQ,\x9e\xc2\xe3\xc9\xa4\xca\xe7\xf3\xf0\xbc\x1cgS\\\xf0\x1f\x82_\xf3\xea\xa6\x9c\xe7tav[,\x96\xf4\xc2y\xb6\xb0\x1fN\x97\xc5\xbc\x98\xe5\xf3\xf0\xf8{^e\xb79}\xdf|\x11|+\xab`\xf1\xa3\xa4\xbbU\x95\xcf\xc6O\xe1\xbbl\x1e\\g7\xd3<\xb8)\xcb\xfbbvk\xae|\x9eM\xe9\xdd`\x92O\x0bz\xfb)|?\xf7\x9f\xe9\x89`V\xfe\x08\xaf~\x14\x8b\xf1]\xb0(\x83\xb2\x9a\xe4U\xf0\x90\xcf\x96\xe1eU\x8c\xf3\x80\xa0\xdc\xe6\xe1\xf1\xedm\x95\xdff\x0b\xfc\xbd\xc0\xf7~1\xf

In [29]:
data = pd.read_csv(r'zomato.csv', encoding = 'ISO-8859-1')
data.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,Average Cost for two,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",1100,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,1200,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",4000,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",1500,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",1500,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


##First observe the data sturcture

In [13]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

###There are various types of restaurants, however we would only focus on one specific type of restaurant at one time. 
###After successful modeling single type restaurant rating, we may extend the model for various types of restaurant.

###We select Fast Food restaurant for it's a common type in the data.

In [56]:
data = data.loc[data['Cuisines'].str.contains('Fast Food').astype(bool)]
data['Cuisines'].head(10)

22                  Fast Food, French
45     Beverages, Bar Food, Fast Food
74          Gourmet Fast Food, Burger
84                                NaN
87                                NaN
88                          Fast Food
94                                NaN
219                         Fast Food
236                         Fast Food
297                               NaN
Name: Cuisines, dtype: object

###Data contains NaN values, we need to drop NaN

In [61]:
data.isna().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [62]:
data.dropna(axis = 0, inplace = True)
print('Current data shape: ', data.shape)
data['Cuisines'].head(10)

Current data shape:  (1987, 21)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


22                  Fast Food, French
45     Beverages, Bar Food, Fast Food
74          Gourmet Fast Food, Burger
88                          Fast Food
219                         Fast Food
236                         Fast Food
394                 Burger, Fast Food
471        Western, Fusion, Fast Food
487               American, Fast Food
525                Fast Food, Seafood
Name: Cuisines, dtype: object

##For our target, observe ratings value:



In [63]:
print(data['Rating text'].value_counts())

Average      987
Not rated    423
Good         415
Very Good     87
Poor          63
Excellent     12
Name: Rating text, dtype: int64


In [64]:
data.groupby('Rating text')['Aggregate rating'].mean()

Rating text
Average      3.051165
Excellent    4.600000
Good         3.670120
Not rated    0.000000
Poor         2.292063
Very Good    4.142529
Name: Aggregate rating, dtype: float64

In [65]:
data = data[data['Rating text'] != 'Not rated']
print(data['Rating text'].value_counts())
print('Current data shape: ', data.shape)

Average      987
Good         415
Very Good     87
Poor          63
Excellent     12
Name: Rating text, dtype: int64
Current data shape:  (1564, 21)


###Since aggregate ratings is positive correlated to Rating text, use Aggregate Rating as target assessment is enough.

###We attempt to model locational data and ratings of restaurants. Geographical data will be import from Foursquare later, so now we need to exclude some redundant feautures from restaurant data first.




In [66]:
columns = ['Restaurant Name', 'City', 'Latitude', 'Longitude', 'Has Table booking', 'Cuisines', 'Has Online delivery', 'Aggregate rating', 'Votes']
data = data[columns]
data.head()

,Restaurant Name,City,Latitude,Longitude,Has Table booking,Cuisines,Has Online delivery,Aggregate rating,Votes
22,Chez Michou,Brasí_lia,-15.764142,-47.881789,No,"Fast Food, French",No,3.0,6
45,Cervantes,Rio de Janeiro,-22.962167,-43.175833,No,"Beverages, Bar Food, Fast Food",No,4.5,29
74,Meats,Sí£o Paulo,-23.564833,-46.681333,No,"Gourmet Fast Food, Burger",No,4.3,68
88,Chick-fil-A,Albany,31.616000,-84.219300,No,Fast Food,No,3.5,67
219,Five Guys Burgers and Fries,Dalton,34.759273,-84.990925,No,Fast Food,No,4.1,142


In [76]:
data['City'].value_counts()

New Delhi          1015
Noida               187
Gurgaon             186
Faridabad            47
Bhopal                8
Allahabad             8
Ghaziabad             8
Kanpur                7
Amritsar              6
Nagpur                6
Mumbai                6
Vizag                 5
Patna                 5
Ahmedabad             4
Indore                4
Coimbatore            4
Ludhiana              3
Manchester            3
Aurangabad            3
Dehradun              3
Ranchi                3
Jaipur                3
Sharjah               3
Vadodara              3
Chandigarh            3
Colombo               3
Lucknow               2
Varanasi              2
Bhubaneshwar          2
Nashik                2
Kochi                 2
Goa                   1
Sí£o Paulo            1
Valdosta              1
Singapore             1
Sioux City            1
Abu Dhabi             1
Wellington City       1
Rio de Janeiro        1
Birmingham            1
Davenport             1
Pocatello       

How many cities do we need to explore? We need to analyze multiple location stores and build prediction model, however too much cities may cost alot of work.